Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploy AzureML FPGA Image to Databox Edge Machine
This notebook will walk through creating and deploying a sample client that can test inferencing of an AzureML FPGA Image. The sample client image contains a script which uses the AzureML SDK to inference and IoT Hub/Device SDK to send messages to your IoT Hub. 

We will use Docker to build the sample client image and push to the Azure Container Registry (ACR) associated with your AzureML Workspace, then use IoT CLI extension to deploy your AzureML Brainwave Image and sample client image to your Databox Edge machine.

 
## Pre-requisites
- You must first have created an AzureML BrainwaveContainerImage using one of the AzureML Brainwave Private Preview notebooks. If you have not, please do so.
    - [Quickstart](https://github.com/Microsoft/aml-fpga-preview/blob/master/notebooks/project-brainwave-quickstart.ipynb)
    - [Custom Weights](https://github.com/Microsoft/aml-fpga-preview/blob/master/notebooks/project-brainwave-custom-weights.ipynb)
    - [Transfer Learning](https://github.com/Microsoft/aml-fpga-preview/blob/master/notebooks/project-brainwave-transfer-learning.ipynb) 
- Follow [these instructions](https://docs.microsoft.com/en-us/azure/databox-online/data-box-edge-deploy-configure-compute) to enable the Linux Compute VM on your Databox Edge Machine.

## Install Azure CLI, IoT extension, and Docker
The following code will:
- install Azure CLI and Azure IoT CLI extension: https://github.com/Azure/azure-iot-cli-extension
- install Docker
- sign into your Azure account using browser login

In [ ]:
%%bash
pip install azure-cli # Install Azure CLI
az extension add --name azure-cli-iot-ext # Add IoT CLI extension
pip install -U jupyter_console # Fix Jupyter dependencies overridden by Azure CLI installation
pip install docker # Install Docker
az login # Log into Azure account

## Set default subscription and resource group

Next, we will set your default subscription and resource group for accessing your IoT Hub, assuming your IoT Hub is in the same subscription and resource group as your AzureML Workspace. You might need to customize these commands if your IoT Hub is in a different subscription or resource group.

In [ ]:
from azureml.core import Workspace
import os

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
os.environ["AZ_SUBSCRIPTION_ID"] = ws.subscription_id
os.environ["AZ_RESOURCE_GROUP"] = ws.resource_group

In [ ]:
%%bash
az account set --subscription $AZ_SUBSCRIPTION_ID
az configure --defaults group=$AZ_RESOURCE_GROUP

## Choose the AzureML FPGA Image

List all Images in your AzureML Workspace.

In [ ]:
from azureml.core.image import Image
for i in Image.list(workspace = ws):
    print('{}'.format(i.image_location))

Choose a converted image you'd like to deploy by copying the entire ACR path below. For example, "workspacename1234.azurecr.io/resnet50.1.brainwaveonnx-image:1".

In [ ]:
os.environ["FPGA_IMAGE_ACR"] = "COPY ACR PATH HERE"
os.environ["ACR_NAME"] = os.environ["FPGA_IMAGE_ACR"].split(".")[0]

print("You are choosing to use the Docker image {} in ACR {}.".format(os.environ["FPGA_IMAGE_ACR"], os.environ["ACR_NAME"]))

## Create Sample Client Image

### Provide IoT Hub and IoT Edge Device information
When setting up your Compute VM for Databox Edge, you created an IoT Hub and Databox Edge automatically created an IoT Edge device registered to your Compute VM. In order to receive messages in your IoT Hub, copy in your IoT Hub Name and IoT Edge Device name to retrieve your connection string.

In [ ]:
%%bash
az iot hub list --output table

In [ ]:
# Update IoT Hub and Edge Device information
os.environ["HUB_NAME"] = "COPY HUB NAME HERE"
# Databox Edge automatically uses your hub name appended with edge as your device name
os.environ["DEVICE_ID"] = "{}-edge".format(os.environ["HUB_NAME"])

os.environ["FILE_PATH"] = "./iotedge-deployment/deployment.json"
os.environ["edgeAgent"] = "$edgeAgent"
os.environ["edgeHub"] = "$edgeHub"

### Build sample client image and push to your ACR

In [ ]:
%%bash
az acr login --name $ACR_NAME
docker build -t $ACR_NAME.azurecr.io/sample-client ./iotedge-deployment/sample-client
docker push $ACR_NAME.azurecr.io/sample-client

## Deploy to Databox Edge Machine
There are two ways to deploy modules to an IoT Edge Device: using the IoT Hub Portal or using the Azure CLI. Here we will utilize the Azure CLI and a pre-defined deployment manifest which will specify how to deploy your IoT Hub modules.

### 1. Deploy using CLI

We will use a deployment.json file which contains all the specifications for IoT Hub host and client module deployment. 

Resources: 
- [az iot edge set-modules](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-deploy-modules-cli)
- [az acr credential show](https://docs.microsoft.com/en-us/cli/azure/acr/credential?view=azure-cli-latest#az-acr-credential-show)
- [az iot hub device-identity show-connection-string](https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-iot-ext/iot/hub/device-identity?view=azure-cli-latest#ext-azure-cli-iot-ext-az-iot-hub-device-identity-show-connection-string)

In [ ]:
%%bash
export ACR_PASSWORD=$(az acr credential show -n $ACR_NAME --query passwords[0].value | grep -oP '"\K[^"\047]+(?=["\047])')
export DEVICE_CONNECTION_STRING=$(az iot hub device-identity show-connection-string --device-id $DEVICE_ID --hub-name $HUB_NAME | grep -oP '(?<=\")HostName(.*)(?=\")')
export TEMPLATE_PATH="./iotedge-deployment/deployment_template.json"

# Replace all environment variables in deployment.json
file=$(<$TEMPLATE_PATH)
echo "$file" | envsubst > $FILE_PATH

In [ ]:
%%bash
# This command sets the deployment.json of your device, which includes the settings for both modules
az iot edge set-modules --device-id $DEVICE_ID --hub-name $HUB_NAME --content $FILE_PATH

### 2. Using IoT Hub Portal
You can also use the IoT Hub Portal to deploy or to check and modify your deployment of the sample client and AzureML container. To specify the Docker container create settings for each module, choose to create an IoT Custom Module and enter the JSON for each module given below into the Create Container Options space.
You can read more about deploying using the IoT Hub Portal [here](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-deploy-modules-portal). 

![Example of IoT Hub Portal Create Container options](./iotedge-deployment/iothub_create_container_options.JPG)


### AzureML Image Container

### Sample Client

## Receive Predictions from IoT Hub
The code below uses the CLI to receive messages from IoT Hub. To use the IoT Service Client SDK, read more here: https://docs.microsoft.com/en-us/azure/iot-hub/iot-hub-devguide-messages-read-builtin

In [ ]:
%%bash
echo $HUB_NAME
echo $DEVICE_ID

#### Open a separate bash window and run the following command, typing in your IoT Hub and device ID.